In [2]:
import pandas as pd

In [3]:
data_ret = pd.read_csv("C:/Users/OS/OneDrive/Máy tính/BTC.csv")
data_label = pd.read_csv("D:/Download/BTCUSDT_label.csv")
label = data_label['LABEL']

In [4]:
# Add a row number column
data_ret['row_number'] = range(1, len(data_ret) + 1)

# Filter rows where the row number is divisible by 12 without remainder
data = data_ret[data_ret['row_number'] % 12 == 0]

In [16]:
import numpy as np
def get_return(data, time1):
    ret = data.copy()
    ret[f'return_{time1}'] = np.log(ret['CLOSE']/ret['CLOSE'].shift(time1))
    return ret

def get_volat(data, time2):
    volat = data.copy()
    volat = get_return(volat,1)
    volat[f"std_{time2}"] = volat[f'return_1'].rolling(window = time2).std()
    volat["std_long"] = volat[f'return_1'].rolling(window = 300).std()
    volat[f"volat_{time2}"] = volat[f"std_{time2}"]/volat["std_long"]
    return volat

In [53]:
import math

sequence = range(1, 27)
ret_dict = {}
for time1 in sequence:
    ret_dict[time1] = get_return(data, time1)

i = 20
w1 = 200
w2 = 200
w3 = 200
w4 = 200
w5 = 200
retur = ret_dict[i][f'return_{i}']*w1 + ret_dict[i+1][f'return_{i+1}']*w2 + ret_dict[i+2][f'return_{i+2}']*w3 + ret_dict[i+3][f'return_{i+3}']*w4 + ret_dict[i+4][f'return_{i+4}']*w5

time2 = 22
volat = get_volat(data,time2)

test = 0.5*retur/(volat['volat_22']**15)
corrr = test.corr(label)
print(corrr)
print(test)

0.022656389586656084
11             NaN
23             NaN
35             NaN
47             NaN
59             NaN
            ...   
612803    0.317138
612815    0.403105
612827    0.518500
612839    0.580302
612851    0.417262
Length: 51071, dtype: float64
